In [3]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] ='3'
import tensorflow as tf
import itertools

# Import models

In [4]:
import joblib
xgb_model = joblib.load('./Models/XGBoost_model.joblib')
xgb_mc_model = joblib.load('./Models/XGBoost_manual_cat_model.joblib')
grb_model = joblib.load('./Models/GRBoost_model.joblib')
lm_model = joblib.load('./Models/LM_model.joblib')
rf_model = joblib.load('./Models/RF_model.joblib')
svr_model = joblib.load('./Models/SVR_model.joblib')
sk_voting_model = joblib.load('./Models/SklearnVoting_model.joblib')
tf_model = tf.keras.models.load_model("./Models/tf_model")

# Data preparation

## Import data with outliers (>= 90 percentile) removed in preprocessing

In [5]:
data = pd.read_csv('../Data/LT_DU_data_for_ML.csv')
data.head()

,nace,esize_class,gender,age_class,lpk,education,experience,target
0,C,1_49,M,40-49,p721,G2,13,8.20
1,C,1_49,F,40-49,p334,G2,0,2.51
2,M,50_249,F,40-49,p522,G2,18,2.19
3,M,50_249,F,40-49,p522,G2,12,2.19
4,M,50_249,F,14-29,p522,G2,0,2.19


## Unique categorical feature values

In [8]:
nace = np.sort(data['nace'].unique())
gender = data['gender'].unique()
esize_class = np.array(['1_49', '50_249','GT_250'], dtype=object)
age_class = np.array(['14-29','30-39', '40-49', '50-59','60+'], dtype=object)
education = np.array([ 'G1', 'G2', 'G3', 'G4'], dtype=object)
lpk = np.sort(data['lpk'].unique())

# Functions to create datasets and apply models

In [10]:
def combination_of_feature_values(nace=nace,
                                esize_class=esize_class,
                                gender=gender,
                                age_class=age_class,
                                lpk=lpk,
                                education=education,
                                experience=[3]):

    """ 
    Inputs should be lists, even if single value is provided e.g [3] for experience
    """

    combination = []
    for t in itertools.product(*[nace, esize_class, gender, age_class, lpk, education, experience]):
        combination.append(t)

    df = pd.DataFrame(combination, columns=['nace', 'esize_class', 'gender', 'age_class', 'lpk', 'education', 'experience'])
    return df

In [11]:
def df_to_dataset(data, shuffle=False, batch_size=256):
  """
  Create TensorFlow dataset from Pandas data frame.
  """
  df = data
  df = {key: value.to_numpy()[:,tf.newaxis] for key, value in data.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df)))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

def apply_models(data): 
  df_ml = data.copy()
  
  df_ml['xgb'] = xgb_model.predict(data)
  df_ml['xgb_mc'] = xgb_mc_model.predict(data)
  df_ml['grb'] = grb_model.predict(data)
  df_ml['rf'] = rf_model.predict(data)
  df_ml['lm'] = lm_model.predict(data)
  df_ml['svr'] = svr_model.predict(data)
  df_ml['voting'] = sk_voting_model.predict(data)

  df_tf = df_to_dataset(data.copy(),  shuffle=False, batch_size=256)
  df_ml['tf'] = tf_model.predict(df_tf)
  
  df_ml_long = pd.melt(
                    df_ml,
                    id_vars=['nace', 'esize_class', 'gender', 'age_class', 'lpk', 'education', 'experience'],
                    var_name='model'
                    )
  return df_ml, df_ml_long

In [69]:
df = combination_of_feature_values(experience=[0, 1, 3, 5, 10, 20])

In [70]:
""" df_ml, df_ml_long = apply_models(df)
df_ml_long.to_csv('../Data/LT_DU_all_feature_combinations_all_model_predictions.csv', index=False) """

5114/5114 [==============================] - 6s 1ms/step
df_ml columns: ['nace' 'esize_class' 'gender' 'age_class' 'lpk' 'education' 'experience'
 'xgb' 'xgb_mc' 'grb' 'rf' 'lm' 'svr' 'voting' 'tf']
df_ml_long columns: ['nace' 'esize_class' 'gender' 'age_class' 'lpk' 'education' 'experience'
 'model' 'value']


# !!! need to reduce file size /5. feather??? less experiance???